In [1]:
import random
import findspark
findspark.init()
import pyspark as ps
from pyspark.sql import SparkSession

In [2]:
spark1 = SparkSession.builder.appName('Ops').getOrCreate()

In [3]:
path = "review.json"
review_df = spark1.read.json(path)
review_df.printSchema()

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)



In [5]:
review_df.createOrReplaceTempView("review")

In [11]:
sample_review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful >= 0")
sample_review_query.show()

+--------+
|count(1)|
+--------+
| 6685899|
+--------+



In [12]:
count_var = sample_review_query.collect()[0][0]
print(count_var)

6685899


In [13]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful >=1")
review_query.show()
useful_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 3115446|
+--------+



In [14]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful =0")
review_query.show()
useless_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 3570453|
+--------+



In [16]:
useful_probability = useful_count / count_var
useless_probability = useless_count / count_var
print("So far for naieve bayes theorum, we have the following probabilities:")
print("probability of a review beeing deemed useful by at least one person P(useful):", useful_probability)
print("probability of a review NOT beeing deemed useful by at least one person P(useless):", useless_probability)

So far for naieve bayes theorum, we have the following probabilities:
probability of a review beeing deemed useful by at least one person P(useful): 0.465972638832863
probability of a review NOT beeing deemed useful by at least one person P(useless): 0.534027361167137


In [ ]:
#continue the conditional probability calculations


In [23]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful >=1 AND funny >= 1")
review_query.show()
useful_funny_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 1169274|
+--------+



In [24]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful >=1 AND cool >= 1")
review_query.show()
useful_cool_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 1449863|
+--------+



In [20]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful =0 AND funny >= 1")
review_query.show()
useless_funny_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
|  204453|
+--------+



In [21]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful =0 AND cool >= 1")
review_query.show()
useless_cool_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
|  222749|
+--------+



In [33]:
#funny and cool reviews
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful >=1 AND funny >=1 AND cool >= 1")
review_query.show()
useful_funny_cool_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
|  873302|
+--------+



In [34]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful =0 AND funny >=1 AND cool >= 1")
review_query.show()
useless_funny_cool_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
|   28852|
+--------+



In [38]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful >=1 AND funny =0")
review_query.show()
useful_not_funny_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 1946172|
+--------+



In [39]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful =0 AND funny =0")
review_query.show()
useless_not_funny_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 3366000|
+--------+



In [41]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful >=1 AND cool =0")
review_query.show()
useful_not_cool_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 1665583|
+--------+



In [42]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful =0 AND cool =0")
review_query.show()
useless_not_cool_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 3347704|
+--------+



In [43]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful >= 1 AND funny = 0 AND cool = 0")
review_query.show()
useful_not_funny_not_cool_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 1369611|
+--------+



In [44]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE useful = 0 AND funny = 0 AND cool = 0")
review_query.show()
useless_not_funny_not_cool_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 3172103|
+--------+



In [ ]:
#my variables so far:

#useful_probability
#useless_probability
#count_var
#useful_count
#useless_count
#useful_funny_count
#useful_cool_count
#useless_funny_count
#useless_cool_count
#useful_funny_cool_count
#useless_funny_cool_count

#useful_not_funny_count
#useless_not_funny_count
#useful_not_cool_count
#useless_not_cool_count
#useful_not_funny_not_cool_count
#useless_not_funny_not_cool_count

In [36]:
#usefull_funny/useless_funny prediction
useful_funny_probability = (useful_funny_count/useful_count)
useless_funny_probability = (useless_funny_count/useless_count)
#usefull_cool/useless_cool prediction
useful_cool_probability = (useful_cool_count/useful_count)
useless_cool_probability = (useless_cool_count/useless_count)
useful_funny_cool_probability = (useful_funny_cool_count/useful_count)
useless_funny_cool_probability = (useless_funny_cool_count/useless_count)
print("P(X|useful)")
print("useful_funny_probability: ", useful_funny_probability)
print("useful_cool_probability: ", useful_cool_probability)
print()
print("P(X|useless)")
print("useless_funny_probability: ", useless_funny_probability)
print("useless_cool_probability: ", useless_cool_probability)
print()
print("accounting for useful/useless probabilities:\n")
print("P(X|useful)*P(useful) and P(X|useless)*P(useless)")
print("probability of a funny review being useful: ", useful_funny_probability * useful_probability)
print("probability of a funny review being useless: ", useless_funny_probability * useless_probability)
print()
print("probability of a cool review being useful: ", useful_cool_probability * useful_probability)
print("probability of a cool review being useless: ", useless_cool_probability * useless_probability)
print()
print("probability of a funny + cool review being useful: ", useful_cool_probability * useful_funny_probability * useful_probability)
print("probability of a funny + cool review being useless: ", useless_cool_probability * useless_funny_probability * useless_probability)
print()
print("now if we aggregate funny and cool as a single class, what is the probability of a funny_cool_review being useful?\n")
print("P(X|useful)")
print("useful_funny_cool_probability: ", useful_funny_cool_probability)
print("P(X|useless)")
print("useless_funny_cool_probability: ", useless_funny_cool_probability)
print("\nP(X|useful)*P(useful) and P(X|useless)*P(useless)")
print("probability of a funny + cool review being useful: ", useful_funny_cool_probability * useful_probability)
print("probability of a funny + cool review being useless: ", useless_funny_cool_probability * useless_probability)

P(X|useful)
useful_funny_probability:  0.37531512342053114
useful_cool_probability:  0.46537895376777516

P(X|useless)
useless_funny_probability:  0.05726248181953383
useless_cool_probability:  0.062386761567789856

accounting for useful/useless probabilities:

P(X|useful)*P(useful) and P(X|useless)*P(useless)
probability of a funny review being useful:  0.17488657845414657
probability of a funny review being useless:  0.030579732059966808

probability of a cool review being useful:  0.21685385914444713
probability of a cool review being useless:  0.033316237651810174

probability of a funny + cool review being useful:  0.08138853290901665
probability of a funny + cool review being useless:  0.0019077704528320486

now if we aggregate funny and cool as a single class, what is the probability of a funny_cool_review being useful?

P(X|useful)
useful_funny_cool_probability:  0.28031363727697417
P(X|useless)
useless_funny_cool_probability:  0.008080767342407253

P(X|useful)*P(useful) and P(

In [ ]:
print("The actual probability of a funny cool review being useful is higher than the predicted baysian \
      model for funny cool review being useful. We believe this to be caused by human nature. If a user \
      is going to check the funny and cool boxes they are more likely to check the useful review box \
      as well. The correlation coefficients for funny and cool reviews are very similar as well which supports this \
      theory.")

In [49]:
#for the sake of completeness, lets look at useful reviews that are neither funny nor cool
#useful_not_funny_count
#useless_not_funny_count
#useful_not_cool_count
#useless_not_cool_count
#useful_not_funny_not_cool_count
#useless_not_funny_not_cool_count
useful_not_funny_probability = useful_not_funny_count/useful_count
useless_not_funny_probability = useless_not_funny_count/useless_count

useful_not_cool_probability = useful_not_cool_count/useful_count
useless_not_cool_probability = useless_not_cool_count/useless_count

#aggregating not cool and not funny into a single class:
useful_not_funny_not_cool_probability = useful_not_funny_not_cool_count/useful_count
useless_not_funny_not_cool_probability = useless_not_funny_not_cool_count/useless_count

print("P(X|useful)*P(useful)")
print("useful_not_funny_probability: ", useful_not_funny_probability * useful_probability)
print("useful_not_cool_probability: ", useful_not_cool_probability * useful_probability)
print("useful but not funny and not cool (classic bayes calculation): ", useful_not_funny_probability * useful_not_cool_probability * useful_probability)
print("useful_not_funny_not_cool_probability (as single class): ", useful_not_funny_not_cool_probability * useful_probability)
print("\nP(X|useless)*P(useless)")
print("useless_not_funny_probability: ", useless_not_funny_probability * useless_probability)
print("useless_not_cool_probability: ", useless_not_cool_probability * useless_probability)
print("useless but not funny and not cool (classic bayes calculation): ", useless_not_funny_probability * useless_not_cool_probability * useless_probability)
print("useless_not_funny_not_cool_probability (as a single class)", useless_not_funny_not_cool_probability * useless_probability)
print()



P(X|useful)*P(useful)
useful_not_funny_probability:  0.2910860603787165
useful_not_cool_probability:  0.24911877968841586
useful but not funny and not cool (classic bayes calculation):  0.15562073414328595
useful_not_funny_not_cool_probability (as single class):  0.20485068649705895

P(X|useless)*P(useless)
useless_not_funny_probability:  0.5034476291071702
useless_not_cool_probability:  0.5007111235153268
useless but not funny and not cool (classic bayes calculation):  0.47203916190819206
useless_not_funny_not_cool_probability (as a single class) 0.4744467423154314



In [52]:
print("testing our model: ")
review_query = spark1.sql("SELECT COUNT(*) from review WHERE funny >=1 AND cool >=1")
review_query.show()
funny_cool_count = review_query.collect()[0][0]


testing our model: 
+--------+
|count(1)|
+--------+
|  902154|
+--------+



In [54]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE funny >=1")
review_query.show()
funny_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 1373727|
+--------+



In [55]:
review_query = spark1.sql("SELECT COUNT(*) from review WHERE cool >=1")
review_query.show()
cool_count = review_query.collect()[0][0]

+--------+
|count(1)|
+--------+
| 1672612|
+--------+



In [67]:
print("lets apply this classification and determine how accurate this model is")
print("funny reviews alone would be classified/assigned to being useful, and the funny count is: ", funny_count, \
      "\nand funny useful count is:", useful_funny_count, \
     "\ngiving us an accuracy of: ", useful_funny_count/funny_count)
print()
print("cool reviews alone would be classified/assigned to being useful, and the cool count is: ", cool_count, \
      "\nand cool useful count is:", useful_cool_count, \
     "\ngiving us an accuracy of: ", useful_cool_count/cool_count)
print()
print("funny cool reviews would be classified/assigned to being useful, and the funny + cool count is: ", funny_cool_count, \
      "\nand funny cool useful count is:", useful_funny_cool_count, \
     "\ngiving us a ratio of: ", useful_funny_cool_count/funny_cool_count)

lets apply this classification and determine how accurate this model is
funny reviews alone would be classified/assigned to being useful, and the funny count is:  1373727 
and funny useful count is: 1169274 
giving us an accuracy of:  0.8511691187550365

cool reviews alone would be classified/assigned to being useful, and the cool count is:  1672612 
and cool useful count is: 1449863 
giving us an accuracy of:  0.8668256595074052

funny cool reviews would be classified/assigned to being useful, and the funny + cool count is:  902154 
and funny cool useful count is: 873302 
giving us a ratio of:  0.9680187639804291


In [68]:
print("Our predictive model indicates that cool and funny reviews are a pretty good \
indicator that a review will be useful. Predicting based on cool reviews alone gives \
a slightly better model then funny alone, but combined provide a very good indication \
that the review will be useful. This phenomenon is most likely explained by if a user \
is going to take the time to vote a review as funny and/or cool, they are very likely \
to vote the review as useful as well, as they have already invested time into voting/reviewing the review")

Our predictive model indicates that cool and funny reviews are a pretty good indicator that a review will be useful. Predicting based on cool reviews alone gives a slightly better model then funny alone, but combined provide a very good indication that the review will be useful. This phenomenon is most likely explained by if a user is going to take the time to vote a review as funny and/or cool, they are very likely to vote the review as useful as well, as they have already invested time into voting/reviewing the review


In [70]:
print("To answer our question of which one is more useful, cool reviews are a marginally \
better indicator that a review will be voted useful compared to funny reviews. If a review \
is voted both funny and cool there is a very strong chance that the review will be useful")

To answer our question of which one is more useful, cool reviews are a marginally better indicator that a review will be voted useful compared to funny reviews. If a review is voted both funny and cool there is a very strong chance that the review will be useful
